# Consolidated Executable Code from DTML_MasterImplementation

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# read the data from the csv file
data = pd.read_csv("C:\\Users\\anshul\\sp3Master.csv")
data.head()

## ---------------------------------------------------------------------------------------------- ##

# everything other than the SR_FLAG is feature vector
# the SR_FLAG is the response variable

# assigning 'SR_FLAG' to the response vector 'y'
y = data['SR_FLAG']

# Using the dataframe.drop() function from the pandas library to 
# remove one column from the data source and create the feature vector set.
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
# the first parameter to the function drop is the name of the column 
# and the second parameter 'axis' indicates whether we are referring to an index or an actual column name:
# 0 or ‘index’, 1 or ‘columns’
X = data.drop('SR_FLAG', axis='columns')

## ---------------------------------------------------------------------------------------------- ##

# split CAL_DATE to numeric values
## SUPER IMPT: This code block only executes once. To reexecute, reset the value of X to default again ##

# convert CAL_DATE to datetime type
X['CAL_DATE'] = pd.to_datetime(X['CAL_DATE'])

# from X, split the date column to individual columns and append them back to X
X_ext = pd.DataFrame({
    # 'CAL_DATE': X['CAL_DATE'],
    'YEAR': X['CAL_DATE'].dt.year,
    'MONTH': X['CAL_DATE'].dt.month,
    'DAY': X['CAL_DATE'].dt.day,
    'DAY_OF_YEAR': X['CAL_DATE'].dt.dayofyear,
    'WEEK': X['CAL_DATE'].dt.week,
    'WEEK_OF_YEAR': X['CAL_DATE'].dt.weekofyear,
    'DAY_OF_WEEK': X['CAL_DATE'].dt.dayofweek,
    'WEEKDAY': X['CAL_DATE'].dt.weekday,
    'QUARTER': X['CAL_DATE'].dt.quarter
}).join(X)
# .join(X, lsuffix='_left')
# }).join(X, on='CAL_DATE', how='right', lsuffix='_left', rsuffix='_right')

# syntax example for join
# df_a.join(df_b, on='mukey', how='left', lsuffix='_left', rsuffix='_right')

# drop the original CAL_DATE column
X_ext = X_ext.drop('CAL_DATE', axis='columns')

# reassign extended X back to X to overrite
X = X_ext

## ---------------------------------------------------------------------------------------------- ##

## CRFA_C
# replace the null values within the column 'CRFA_C' with ASCII for '0' that is NUL.
X['CRFA_C'].fillna(chr(0), inplace=True)

# convert all the string values to ASCII equivalents
X['CRFA_C'] = X['CRFA_C'].apply(lambda x: ord(x))


##OUTPROC_FLAG
# convert the boolean values True/False to integer 1/0
X['OUTPROC_FLAG']=X['OUTPROC_FLAG'].apply(lambda x: np.where(x==False,0,1))

## hence the only non-numeric column that is left now is ITEM_NO


## ---------------------------------------------------------------------------------------------- ##

# collect all the numeric features in a temp dataframe
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.select_dtypes.html
X_numeric = X.select_dtypes(include=np.number)

# fetch the column names of the numeric columns
X_numcols = list(X_numeric)
print(X_numcols)
# use the apply function again to apply logorithmic converion to all numeric columns
# since log of negative values is not defined, we validate and change the function for negative values
# X[X_numcols] = X[X_numcols].apply(lambda x: (np.log(1 + x) if x > 0 else -np.log(1 - x)))
# X[X_numcols] = X[X_numcols].apply(lambda x: np.log(1 + x))

X[X_numcols] = X[X_numcols].apply(lambda x: ((1+x)/(1+abs(x)))*(np.log(1 + abs(x))))

## ---------------------------------------------------------------------------------------------- ##

# Again, scaling only applies to numerical data.
from sklearn.preprocessing import RobustScaler

# initializing the scaler object
scaler = RobustScaler()

# define a new dataframe to hold the scaled numeric features
X_scaled = pd.DataFrame(data = X)

# apply the scaling and populate the data
X_scaled[X_numcols] = scaler.fit_transform(X[X_numcols])

X_scaled.head()

## ---------------------------------------------------------------------------------------------- ##

# The 'pandas.get_dummies' is a function to convert string columns into one-hot representation.
# Alternative is sklearn.preprocessing.OneHotEncoder()
# https://stackoverflow.com/questions/36631163/pandas-get-dummies-vs-sklearns-onehotencoder-what-is-more-efficient

# count of features before applying one-hot encoding:
print('Count of features before one-hot-encoding: ', len(X_scaled.columns))

# applying one-hot-encoding
# One-hot encoding is failing the modle fitment
X_scaled = pd.get_dummies(data=X_scaled)

# count of features after applying one-hot encoding:
print('Count of features after one-hot-encoding: ', len(X_scaled.columns))

## ---------------------------------------------------------------------------------------------- ##

# split the scaled dataset into a training and testing set
from sklearn.model_selection import train_test_split

# splitting the data in to groups of 75/25 as train/test ratio with a fixed random_state of 0.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=0)

# check the types of the records
print('X_train Type: ', type(X_train))
print('X_test Type: ', type(X_test))
print('y_train Type: ', type(y_train))
print('y_test Type: ', type(y_test))

# check the shapes of the records
print('X_train Shape: ', X_train.shape)
print('X_test Shape: ', X_test.shape)
print('y_train Shape: ', y_train.shape)
print('y_test Shape: ', y_test.shape)

# Using LabelBinarizer to convert y_train and y_test to a binary array
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_trbin = lb.fit_transform(y_train)
y_tsbin = lb.fit_transform(y_test)
print('y_trbin Type: ', type(y_trbin))
print('y_tsbin Type: ', type(y_tsbin))

# convert the numpy array back to the pandas series
y_trbin = pd.Series(y_trbin.reshape(-1))
y_tsbin = pd.Series(y_tsbin.reshape(-1))
print('y_trbin Type: ', type(y_trbin))
print('y_trbin Shape: ', y_trbin.shape)
print('y_tsbin Type: ', type(y_tsbin))
print('y_tsbin Shape: ', y_tsbin.shape)

## ---------------------------------------------------------------------------------------------- ##

# Calculating the naive parameters for the data
# Pre-calculating the metrices for a naive model. 
# That is when the outcome is always predicted as true, how will the raw model behave.
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, matthews_corrcoef, classification_report

# predicting everything as True(1)
y_naive = [1]*len(y_test)

print('Accuracy: ', accuracy_score(y_naive, y_test))
print('Recall: ', recall_score(y_naive, y_test))
print('Precision: ', precision_score(y_naive, y_test))
print('F1-Score: ', f1_score(y_naive, y_test))
# print('Matthews Correlation Coefficient: ', matthews_corrcoef(y_naive, y_test))
print('Classification Report: \n', classification_report(y_naive, y_test, target_names=['Short'], labels=[1]))
# print ("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

## ---------------------------------------------------------------------------------------------- ##

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, fbeta_score

# clf = LogisticRegression()

## GridSearchCV is throwing a MemoryError ##

# attributes for GridSearchCV
# parameters for the attribute 'param_grid' ## BALCK BOX ##
# parameters = {'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced']}
# parameters = {'C': [10], 'class_weight': [None, 'balanced']}
# parameters = {'C': [10]}
# scorer for the attribute 'scoring' ## BALCK BOX ##
# scorer = make_scorer(fbeta_score, beta=0.5)

# create an object for GridSearchCV
# grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=scorer)

# fitting the training dataset
# grid_fit = grid_obj.fit(X_train, y_train)

# get the best estimator
# best_clf = grid_fit.best_estimator_

# default prediction - without considering the recommendations from GridSearchCV
# fitting the model
# clf.fit(X_train, y_train)
# making predictions
# y_pred = clf.predict(X_test)

# predictions based on the recommendations via GridSearchCV
# best_y_pred = best_clf.predict(X_test)

## ---------------------------------------------------------------------------------------------- ##



['DAY', 'DAY_OF_WEEK', 'DAY_OF_YEAR', 'MONTH', 'QUARTER', 'WEEK', 'WEEKDAY', 'WEEK_OF_YEAR', 'YEAR', 'LOC', 'CRFA_R', 'CRFA_F', 'CRFA_A', 'EOQ', 'PKG_QTY', 'SCORE_862_INSTABILITY_AVG', 'AVG_ADJ_QTY', 'DESTINID', 'AVG_DAYS_LATE']
Count of features before one-hot-encoding:  22
Count of features after one-hot-encoding:  22
X_train Type:  <class 'pandas.core.frame.DataFrame'>
X_test Type:  <class 'pandas.core.frame.DataFrame'>
y_train Type:  <class 'pandas.core.series.Series'>
y_test Type:  <class 'pandas.core.series.Series'>
X_train Shape:  (77589, 22)
X_test Shape:  (25864, 22)
y_train Shape:  (77589,)
y_test Shape:  (25864,)
y_trbin Type:  <class 'numpy.ndarray'>
y_tsbin Type:  <class 'numpy.ndarray'>
y_trbin Type:  <class 'pandas.core.series.Series'>
y_trbin Shape:  (77589,)
y_tsbin Type:  <class 'pandas.core.series.Series'>
y_tsbin Shape:  (25864,)
Accuracy:  0.2992189916486236
Recall:  0.2992189916486236
Precision:  1.0
F1-Score:  0.4606136356872898
Classification Report: 
          

In [57]:
# identify the non numeric features
# collect all the numeric features in a temp dataframe
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.select_dtypes.html
# X_nonnum = X_scaled.select_dtypes(exclude=np.number)
# fetch the column names of the numeric columns
# X_nnumcols = list(X_nonnum)
# print(X_nnumcols)
# use the apply function again to apply logorithmic converion to all numeric columns
# since log of negative values is not defined, we validate and change the function for negative values
# X[X_numcols] = X[X_numcols].apply(lambda x: (np.log(1 + x) if x > 0 else -np.log(1 - x)))
# X[X_numcols] = X[X_numcols].apply(lambda x: np.log(1 + x))

# use DictVectorizer to convert these colums to numeric fields
# check unique CRFA_C counts
# X_uniq = X_scaled.groupby('OUTPROC_FLAG').nunique()
# print(X_uniq)

# convert 'CRFA_C' to ASCII values using the Python ord() function
# X_scaled['CRFA_C'] = X_scaled['CRFA_C'].apply(lambda x: ord(x))

X_nonnum = X_scaled.select_dtypes(exclude=np.number)
X_nnumcols = list(X_nonnum)
# print(X_nnumcols)


# X_log_scaled['OUTPROC_FLAG']=X_log_scaled['OUTPROC_FLAG'].apply(lambda x: np.where(x==False,0,1))
# ord('S')

# X_scaled['CRFA_C'].fillna('0', inplace=True)
print(X_scaled['CRFA_C'].unique())
'''
l_temp = ['S', '2', '3', 'K', 'D', 'G', 'B', 'E', 'H', 'Z', 'F', nan]
l_temp2 = [0]*len(l_temp)
for i in range(len(l_temp)):
    if len(str(l_temp[i])) != 1:
        l_temp2[i] = 0 #if l_temp[i].isalpha() else ord(chr(l_temp[i]))
    else:
        l_temp2[i] = ord(l_temp[i])
print(l_temp2)
'''
# X_scaled['CRFA_C'] = X_scaled['CRFA_C'].apply(lambda x: np.where(x.isalpha(), ord(x), ord(chr(x))))
# X_scaled = X_scaled[X_scaled['CRFA_C'].notnull()]
# X_scaled['CRFA_C'] = X_scaled['CRFA_C'].apply(lambda x: ord(x))

# X_scaled['CRFA_C']

['ITEM_NO', 'OUTPROC_FLAG']


"\nl_temp = ['S', '2', '3', 'K', 'D', 'G', 'B', 'E', 'H', 'Z', 'F', nan]\nl_temp2 = [0]*len(l_temp)\nfor i in range(len(l_temp)):\n    if len(str(l_temp[i])) != 1:\n        l_temp2[i] = 0 #if l_temp[i].isalpha() else ord(chr(l_temp[i]))\n    else:\n        l_temp2[i] = ord(l_temp[i])\nprint(l_temp2)\n"

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# read the data from the csv file
data = pd.read_csv("C:\\Users\\anshul\\sp3Master.csv")
data.head()

## ---------------------------------------------------------------------------------------------- ##

# everything other than the SR_FLAG is feature vector
# the SR_FLAG is the response variable

# assigning 'SR_FLAG' to the response vector 'y'
y = data['SR_FLAG']

# Using the dataframe.drop() function from the pandas library to 
# remove one column from the data source and create the feature vector set.
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
# the first parameter to the function drop is the name of the column 
# and the second parameter 'axis' indicates whether we are referring to an index or an actual column name:
# 0 or ‘index’, 1 or ‘columns’
X = data.drop('SR_FLAG', axis='columns')

## ---------------------------------------------------------------------------------------------- ##

# split CAL_DATE to numeric values
## SUPER IMPT: This code block only executes once. To reexecute, reset the value of X to default again ##

# convert CAL_DATE to datetime type
X['CAL_DATE'] = pd.to_datetime(X['CAL_DATE'])

# from X, split the date column to individual columns and append them back to X
X_ext = pd.DataFrame({
    # 'CAL_DATE': X['CAL_DATE'],
    'YEAR': X['CAL_DATE'].dt.year,
    'MONTH': X['CAL_DATE'].dt.month,
    'DAY': X['CAL_DATE'].dt.day,
    'DAY_OF_YEAR': X['CAL_DATE'].dt.dayofyear,
    'WEEK': X['CAL_DATE'].dt.week,
    'WEEK_OF_YEAR': X['CAL_DATE'].dt.weekofyear,
    'DAY_OF_WEEK': X['CAL_DATE'].dt.dayofweek,
    'WEEKDAY': X['CAL_DATE'].dt.weekday,
    'QUARTER': X['CAL_DATE'].dt.quarter
}).join(X)
# .join(X, lsuffix='_left')
# }).join(X, on='CAL_DATE', how='right', lsuffix='_left', rsuffix='_right')

# syntax example for join
# df_a.join(df_b, on='mukey', how='left', lsuffix='_left', rsuffix='_right')

# drop the original CAL_DATE column
X_ext = X_ext.drop('CAL_DATE', axis='columns')

# reassign extended X back to X to overrite
X = X_ext

## ---------------------------------------------------------------------------------------------- ##

## CRFA_C
# replace the null values within the column 'CRFA_C' with ASCII for '0' that is NUL.
X['CRFA_C'].fillna(chr(0), inplace=True)

# convert all the string values to ASCII equivalents
X['CRFA_C'] = X['CRFA_C'].apply(lambda x: ord(x))


##OUTPROC_FLAG
# convert the boolean values True/False to integer 1/0
X['OUTPROC_FLAG']=X['OUTPROC_FLAG'].apply(lambda x: np.where(x==False,0,1))

## hence the only non-numeric column that is left now is ITEM_NO


## ---------------------------------------------------------------------------------------------- ##

# collect all the numeric features in a temp dataframe
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.select_dtypes.html
X_numeric = X.select_dtypes(include=np.number)

# fetch the column names of the numeric columns
X_numcols = list(X_numeric)
print(X_numcols)
# use the apply function again to apply logorithmic converion to all numeric columns
# since log of negative values is not defined, we validate and change the function for negative values
# X[X_numcols] = X[X_numcols].apply(lambda x: (np.log(1 + x) if x > 0 else -np.log(1 - x)))
# X[X_numcols] = X[X_numcols].apply(lambda x: np.log(1 + x))

X[X_numcols] = X[X_numcols].apply(lambda x: ((1+x)/(1+abs(x)))*(np.log(1 + abs(x))))

## ---------------------------------------------------------------------------------------------- ##

# Again, scaling only applies to numerical data.
from sklearn.preprocessing import RobustScaler

# initializing the scaler object
scaler = RobustScaler()

# define a new dataframe to hold the scaled numeric features
X_scaled = pd.DataFrame(data = X)

# apply the scaling and populate the data
X_scaled[X_numcols] = scaler.fit_transform(X[X_numcols])

X_scaled.head()

## ---------------------------------------------------------------------------------------------- ##

# The 'pandas.get_dummies' is a function to convert string columns into one-hot representation.
# Alternative is sklearn.preprocessing.OneHotEncoder()
# https://stackoverflow.com/questions/36631163/pandas-get-dummies-vs-sklearns-onehotencoder-what-is-more-efficient

# count of features before applying one-hot encoding:
print('Count of features before one-hot-encoding: ', len(X_scaled.columns))

# applying one-hot-encoding
# One-hot encoding is failing the modle fitment
X_scaled = pd.get_dummies(data=X_scaled)

# count of features after applying one-hot encoding:
print('Count of features after one-hot-encoding: ', len(X_scaled.columns))

## ---------------------------------------------------------------------------------------------- ##

['DAY', 'DAY_OF_WEEK', 'DAY_OF_YEAR', 'MONTH', 'QUARTER', 'WEEK', 'WEEKDAY', 'WEEK_OF_YEAR', 'YEAR', 'LOC', 'CRFA_C', 'CRFA_R', 'CRFA_F', 'CRFA_A', 'EOQ', 'PKG_QTY', 'OUTPROC_FLAG', 'SCORE_862_INSTABILITY_AVG', 'AVG_ADJ_QTY', 'DESTINID', 'AVG_DAYS_LATE']
Count of features before one-hot-encoding:  22
Count of features after one-hot-encoding:  11421


In [61]:
print(X['CRFA_C'].unique())

[83 50 51  0 75 68 71 66 69 72 90 70]


In [63]:
X_nonnum = X.select_dtypes(exclude=np.number)
X_nnumcols = list(X_nonnum)
print(X_nnumcols)

['ITEM_NO']
